In [81]:
from sympy import *
init_printing(order='none')
from IPython.display import Markdown

def display_expr(expr, label="", uneval_exprs=[]):
    wild = Wild("wild")
    wild2 = Wild("wild2")
    for e in uneval_exprs:
        expr = expr.subs(e, UnevaluatedExpr(e))
        expr = expr.replace(UnevaluatedExpr(wild)**wild2, UnevaluatedExpr(UnevaluatedExpr(wild)**wild2))
        expr = expr.replace(wild*UnevaluatedExpr(wild2), UnevaluatedExpr(wild)*UnevaluatedExpr(wild2))
    return display(Markdown(f'<div style="text-align: center">\n\n#### {label}\n\n</div>\n\n' + '\\begin{equation}\n' + f'{latex(expr)}' + ' \\end{equation}'))

# Feasibility of a Vacuum Airship

A [vacuum airship](https://en.wikipedia.org/wiki/Vacuum_airship) is a structure that is less dense than air (at [STP](https://en.wikipedia.org/wiki/Standard_temperature_and_pressure)) but does not contain gases that are (or would be) less dense than air at STP.

The yield stress of the ship $\sigma_{ship}$ must be at least the air pressure $\sigma_{air}$:

In [82]:
sigma_air = symbols("\\sigma_{air}", real=True, finite=True, positive=True)
sigma_ship = symbols("\\sigma_{ship}", real=True, finite=True, positive=True)


display_expr(Ge(sigma_ship, sigma_air))

<div style="text-align: center">

#### 

</div>

\begin{equation}
\sigma_{ship} \geq \sigma_{air} \end{equation}


To ensure bouyancy, the density of the ship $\rho_{ship}$ must be less than the density of the air $\rho_{air}$:

In [83]:
rho_ship = symbols("\\rho_{ship}", real=True, finite=True, positive=True)
rho_air = symbols("\\rho_{air}", real=True, finite=True, positive=True)
display_expr(Lt(rho_ship, rho_air))

<div style="text-align: center">

#### 

</div>

\begin{equation}
\rho_{ship} < \rho_{air} \end{equation}

So:

## Constraints imposed if using an octet truss

Since almost all structural materials have a higher density than air we will need to build a structure that has less density than its material but still has sufficient strengh. In this section, we consider forming the material into an octet truss structure. If each unit of the octet truss is less dense than air is strong enough to resist air pressure, a "solid" structure made from a sufficiently large number of the units and covered with an airtight skin would float. The idea is that the mass of the airtight skin only increases with the surface area of the structure, while the overall mass increases with it's volume. As a result the relative contribution of the mass of the skin is neglible for a sufficiently large structure.


If the struts of the octet-truss are hollow tubes of diameter $d$ with a wall thickness of $t$, made from a material with compressive yield strength of $\sigma_m$ and density $\rho_m$, the density can be lowered for an equal drop in strength. For $t \ll d$:

In [84]:
d = symbols("d", real=True, finite=True, positive=True)
t = symbols("t", real=True, finite=True, positive=True)
rho_m = symbols("\\rho_m", real=True, finite=True, positive=True)
sigma_m = symbols("\\sigma_m", real=True, finite=True, positive=True)
rho_strut = symbols("\\rho_{strut}", real=True, finite=True, positive=True)
eq_rho_strut = Eq(rho_strut, 2*t/d*rho_m)
display_expr(eq_rho_strut, "", [2*t/d])
sigma_strut = symbols("\\sigma_{strut}", real=True, finite=True, positive=True)
eq_sigma_strut = Eq(sigma_strut, 2*t/d*sigma_m)
display_expr(eq_sigma_strut, "", [2*t/d])


<div style="text-align: center">

#### 

</div>

\begin{equation}
\rho_{strut} = \rho_{m} \frac{2 t}{d} \end{equation}

<div style="text-align: center">

#### 

</div>

\begin{equation}
\sigma_{strut} = \sigma_{m} \frac{2 t}{d} \end{equation}


Formulas which relate the density and strength of an octet truss to the density and strength of the strut material as well as the slenderness ratio of the struts can be found in section 2.1.1 of [a 2020 paper by Lijun Xiao, et al](https://www.researchgate.net/publication/344302750_A_Multi-Cell_Hybrid_Approach_to_Elevate_the_Energy_Absorption_of_Micro-Lattice_Materials). Specifically, if $d$ and $l$ are the diameter and length of a strut respectively, $d \ll l$, but $d/l$ is not so small that the struts buckle before yielding:

In [85]:
rho_oct = symbols("\\rho_{oct}", real=True, finite=True, positive=True)
l = symbols("l", real=True, finite=True, positive=True)
eq_rho_oct_strut = Eq(rho_oct/rho_strut, 3*sqrt(2)*pi/2*(d/l)**2)
display_expr(eq_rho_oct_strut, "", [d/l])

sigma_oct = symbols("\\sigma_{oct}", real=True, finite=True, positive=True)
eq_sigma_oct_strut = Eq(sigma_oct/sigma_strut, rho_oct/(3*rho_strut))
display_expr(eq_sigma_oct_strut, "", [rho_oct/rho_strut])


<div style="text-align: center">

#### 

</div>

\begin{equation}
\frac{\rho_{oct}}{\rho_{strut}} = \frac{3 \pi \sqrt{2}}{2} \left(\frac{d}{l}\right)^{2} \end{equation}

<div style="text-align: center">

#### 

</div>

\begin{equation}
\frac{\sigma_{oct}}{\sigma_{strut}} = \frac{1}{3} \frac{\rho_{oct}}{\rho_{strut}} \end{equation}

Combining those formulas with the formulas for the density and strength of the strut gives:

In [86]:
eq_rho_oct = Eq(rho_oct, solve([eq_rho_oct_strut, eq_rho_strut], [rho_oct, rho_strut], dict=True)[0][rho_oct])
eq_rho_oct = use(eq_rho_oct, lambda side: side/rho_m, level=1)
display_expr(eq_rho_oct, "", [d*t/l**2])

eq_sigma_oct = eq_sigma_oct_strut.subs(eq_sigma_strut.lhs, eq_sigma_strut.rhs).subs(eq_rho_strut.lhs, eq_rho_strut.rhs)
eq_sigma_oct = use(eq_sigma_oct, lambda side: side*2*t/d, level=1)
display_expr(eq_sigma_oct, "", [rho_oct/rho_m])


<div style="text-align: center">

#### 

</div>

\begin{equation}
\frac{\rho_{oct}}{\rho_{m}} = 3 \pi \sqrt{2} \frac{d t}{l^{2}} \end{equation}

<div style="text-align: center">

#### 

</div>

\begin{equation}
\frac{\sigma_{oct}}{\sigma_{m}} = \frac{1}{3} \frac{\rho_{oct}}{\rho_{m}} \end{equation}

Note that the yield strength of the octet-truss $\sigma_{oct}$ must be at least the pressure exerted by the $\sigma_{air}$, and the density of the octet-truss $\rho_{oct}$ must be less than the density of air $\rho_{air}$:

In [87]:
display_expr(Ge(sigma_oct, sigma_air))
display_expr(Lt(rho_oct, rho_air))


<div style="text-align: center">

#### 

</div>

\begin{equation}
\sigma_{oct} \geq \sigma_{air} \end{equation}

<div style="text-align: center">

#### 

</div>

\begin{equation}
\rho_{oct} < \rho_{air} \end{equation}

We can combine these two inequalities to get:

In [88]:
specific_strength_oct_constraint = Ge(sigma_oct/rho_oct, sigma_air/rho_air)
display_expr(specific_strength_oct_constraint)

<div style="text-align: center">

#### 

</div>

\begin{equation}
\frac{\sigma_{oct}}{\rho_{oct}} \geq \frac{\sigma_{air}}{\rho_{air}} \end{equation}

Combining that with the equation for the relative strength of the octet truss gives:

In [89]:
# Solve eq_sigma_oct for sigma_oct and substitute that into specific_strength_oct_constraint
specific_strength_constraint = specific_strength_oct_constraint.subs(sigma_oct, solve([eq_sigma_oct], [sigma_oct], dict=True)[0][sigma_oct])
# Multiply both sides by 3
specific_strength_constraint = use(specific_strength_constraint, lambda s: 3*s, level=1)
display_expr(specific_strength_constraint, "", [sigma_air/rho_air])

<div style="text-align: center">

#### 

</div>

\begin{equation}
\frac{\sigma_{m}}{\rho_{m}} \geq 3 \frac{\sigma_{air}}{\rho_{air}} \end{equation}

Substituting the values for air at standard temperature and pressure, gives:

In [90]:
specific_strength_constraint_stp = specific_strength_constraint.subs(sigma_air, 101000).subs(rho_air, 1.225)
display_expr(N(specific_strength_constraint_stp, 4), "Specific Strength Constraint")

<div style="text-align: center">

#### Specific Strength Constraint

</div>

\begin{equation}
\frac{\sigma_{m}}{\rho_{m}} \geq 2.473 \cdot 10^{5} \end{equation}

Note however, that there are limits to how small $2t/d$ or $d/l$ can be. Various references indicate that to avoid thin-wall effects $2t/d > 1/10$, and to ensure that the strut doesn't collapse due to buckling before a material with Young's modulus $E_m$ yields:

In [91]:
E_m = symbols("E_m", real=True, finite=True, positive=True)
yield_before_buckling_constraint = Lt(sigma_m, pi**2*E_m/(2*l/d)**2)
display_expr(yield_before_buckling_constraint, "", [d/l])

<div style="text-align: center">

#### 

</div>

\begin{equation}
\sigma_{m} < \frac{E_{m} \pi^{2}}{4} \left(\frac{d}{l}\right)^{2} \end{equation}

Rearranging that results in a slenderness constraint:

In [92]:

slenderness_constraint = use(yield_before_buckling_constraint, lambda side: side/(E_m*pi**2/4), level=1)
display_expr(slenderness_constraint, "", [d/l])


<div style="text-align: center">

#### 

</div>

\begin{equation}
\frac{4 \sigma_{m}}{\pi^{2} E_{m}} < \left(\frac{d}{l}\right)^{2} \end{equation}

Combining that constraint and the constraint to avoid thin-walls gives:


In [93]:
geometry_constraint = Lt(slenderness_constraint.lhs/10, slenderness_constraint.rhs*2*t/d)
geometry_constraint = use(geometry_constraint, lambda side: side/2, level=1)
display_expr(geometry_constraint)

<div style="text-align: center">

#### 

</div>

\begin{equation}
\frac{\sigma_{m}}{5 \pi^{2} E_{m}} < \frac{d t}{l^{2}} \end{equation}

Solving the relative density formula for the octet-truss for the density of the truss gives:

In [94]:
eq_rho_oct_solved = Eq(rho_oct, solve([eq_rho_oct], [rho_oct], dict=True)[0][rho_oct])
display_expr(eq_rho_oct_solved)

<div style="text-align: center">

#### 

</div>

\begin{equation}
\rho_{oct} = \frac{3 \pi \rho_{m} d t \sqrt{2}}{l^{2}} \end{equation}

Combining with the geometry constraint and the fact that $\rho_{oct} < \rho_{air}$ buoyancy, gives:

In [95]:
density_constraint = Gt(rho_air, eq_rho_oct_solved.rhs.subs(geometry_constraint.rhs, geometry_constraint.lhs))
display_expr(density_constraint)

<div style="text-align: center">

#### 

</div>

\begin{equation}
\rho_{air} > \frac{3 \rho_{m} \sigma_{m} \sqrt{2}}{5 \pi E_{m}} \end{equation}

Rearranging so that all of the material properties are on one side gives a constraint on the the product of the material's density $\rho_m$ and the material's yield strain (which is the $\sigma_m/E_m$):

In [96]:
density_strain_constraint = Lt(rho_m*sigma_m/E_m, solve([Eq(density_constraint.lhs, density_constraint.rhs)], [rho_m*sigma_m/E_m], dict=True)[0][rho_m*sigma_m/E_m])
display_expr(density_strain_constraint, "", [sigma_m/E_m, rho_air])


<div style="text-align: center">

#### 

</div>

\begin{equation}
\rho_{m} \frac{\sigma_{m}}{E_{m}} < \frac{5 \pi \sqrt{2}}{6} \rho_{air} \end{equation}

Using the density of air at STP:

In [97]:
density_strain_constraint_stp = density_strain_constraint.subs(rho_air, 1.225)
display_expr(N(density_strain_constraint_stp, 4), "Density Strain Constraint", [sigma_m/E_m])


<div style="text-align: center">

#### Density Strain Constraint

</div>

\begin{equation}
\rho_{m} \frac{\sigma_{m}}{E_{m}} < 4.535 \end{equation}

Unfortunately, none of the following materials seems to meet all of the constraints:

In [98]:
specific_strength = specific_strength_constraint_stp.lhs
density_strain = density_strain_constraint_stp.lhs

table = f"""
| Material    | $\\sigma_m (MPa)$ | $E_m (MPa)$ | $\\rho_m (kg/m^3)$ | Specific Strength $(kPa/kg)$ | Density Strain $(kg/m^3)$ |
| ----------- | ---------------- | ---------- | ------------------  | --------------------------- | ------------------------- |
| *Constraint*|                  |            |                    | > {N(specific_strength_constraint_stp.rhs/1000, 4)} | < {N(density_strain_constraint_stp.rhs, 4)} |"""

materials = [
    { 'name': "Graphene aerogel", E_m: 50e6, sigma_m: 0.5*50e6, rho_m: .053e3 },
    { 'name': "CFRP High Modulus Unidirectional", E_m: 175e9, sigma_m: 850e6, rho_m: 1.6e3 },
    { 'name': "CFRP Standard Unidirectional", E_m: 135e9, sigma_m: 1200e6, rho_m: 1.6e3 },
    { 'name': "CFRP Standard +/-45 deg", E_m: 17e9, sigma_m: 110e6, rho_m: 1.6e3 },
    { 'name': "Diamond", E_m: 1220e9, sigma_m: 110e9, rho_m: 3.52e3 },
    { 'name': "Aluminum 7075-T6", E_m: 71.7e9, sigma_m: 503e6, rho_m: 2.81e3 },
    { 'name': "Boron Unidirectional", E_m: 200e9, sigma_m: 2800e6, rho_m: 2.00e3 },
]

for m in materials:
    m["specific_strength"] = specific_strength.evalf(subs=m)
    m["density_strain"] = density_strain.evalf(subs=m)

    table += f"""
| {m['name']} | {N(m[sigma_m]/1e6, 4)}     | {N(m[E_m]/1e6, 4)}   | {m[rho_m]}         | {N(m['specific_strength']/1000, 4)}    | {N(m['density_strain'], 4)}     |"""

display(Markdown(table))



| Material    | $\sigma_m (MPa)$ | $E_m (MPa)$ | $\rho_m (kg/m^3)$ | Specific Strength $(kPa/kg)$ | Density Strain $(kg/m^3)$ |
| ----------- | ---------------- | ---------- | ------------------  | --------------------------- | ------------------------- |
| *Constraint*|                  |            |                    | > 247.3 | < 4.535 |
| Graphene aerogel | 25.00     | 50.00   | 53.0         | 471.7    | 26.50     |
| CFRP High Modulus Unidirectional | 850.0     | 1.750E+5   | 1600.0         | 531.3    | 7.771     |
| CFRP Standard Unidirectional | 1200     | 1.350E+5   | 1600.0         | 750.0    | 14.22     |
| CFRP Standard +/-45 deg | 110.0     | 1.700E+4   | 1600.0         | 68.75    | 10.35     |
| Diamond | 1.100E+5     | 1.220E+6   | 3520.0         | 3.125E+4    | 317.4     |
| Aluminum 7075-T6 | 503.0     | 7.170E+4   | 2810.0         | 179.0    | 19.71     |
| Boron Unidirectional | 2800     | 2.000E+5   | 2000.0         | 1400    | 28.00     |